In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.8/702.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
import time
from math import dist
model=YOLO('yolov8l.pt')



100%|██████████| 83.7M/83.7M [00:00<00:00, 263MB/s]


In [ ]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :
        colorsBGR = [x, y]
        print(colorsBGR)


#cv2.namedWindow('RGB')
#cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture(r'/content/cartraffic1.mp4')


my_file = open('/content/coco.txt', "r")
video_path = '/content/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_size = (1020,500)
out = cv2.VideoWriter(video_path, fourcc, 20.0, frame_size)
data = my_file.read()
class_list = data.split("\n")
#print(class_list)

count=0

tracker=Tracker()

cy1=322
cy2=368

offset=6

vh_down={}
counter=[]


vh_up={}
counter1=[]

while True:
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))


    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]

    for index,row in px.iterrows():
#        print(row)

        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)


        print("bbox:",bbox)
        if cy1<(cy+offset) and cy1 > (cy-offset):
            print("vh down:",id)
            vh_down[id]=time.time()
        if id in vh_down:
            print("vh down1:",id)
            if cy2<(cy+offset) and cy2 > (cy-offset):
                elapsed_time=time.time() - vh_down[id]
                print("vh down2:",id,counter)
                if counter.count(id)==0:
                    counter.append(id)
                    distance = 10 # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                    cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                    cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


        #####going UP#####
        if cy2<(cy+offset) and cy2 > (cy-offset):
            print("vh up:",id)
            vh_up[id]=time.time()
        if id in vh_up:
            print("vh up1:",id)
            if cy1<(cy+offset) and cy1 > (cy-offset):
                print("vh up2:",id)
                elapsed1_time=time.time() - vh_up[id]
                if counter1.count(id)==0:
                    counter1.append(id)
                    distance1 = 10 # meters
                    a_speed_ms1 = distance1 / elapsed1_time
                    a_speed_kh1 = a_speed_ms1 * 3.6
                    cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                    cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                    cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    cv2.line(frame,(274,cy1),(814,cy1),(255,255,255),1)

    cv2.putText(frame,('L1'),(277,320),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


    cv2.line(frame,(177,cy2),(927,cy2),(255,255,255),1)

    cv2.putText(frame,('L2'),(182,367),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    d=(len(counter))
    u=(len(counter1))
    cv2.putText(frame,('goingdown:-')+str(d),(60,90),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    cv2.putText(frame,('goingup:-')+str(u),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.putText(frame,str(id)+c,(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
    #cv2.imshow("RGB", frame)
    #if cv2.waitKey(1)&0xFF==27:
       # break
    out.write(frame)

cap.release()
#cv2.destroyAllWindows()
out.release()


0: 320x640 19 cars, 1 bus, 2 trucks, 1345.3ms
Speed: 4.7ms preprocess, 1345.3ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)
bbox: [177, 284, 343, 413, 0]
bbox: [516, 242, 660, 360, 1]
bbox: [288, 210, 421, 296, 2]
bbox: [0, 142, 87, 201, 3]
bbox: [272, 272, 394, 376, 4]
vh down: 4
vh down1: 4
bbox: [482, 181, 578, 253, 5]
bbox: [530, 165, 610, 219, 6]
bbox: [306, 173, 396, 222, 7]
bbox: [529, 218, 639, 267, 8]
bbox: [351, 159, 425, 221, 9]
bbox: [504, 142, 577, 184, 10]
bbox: [124, 410, 277, 446, 11]
bbox: [331, 137, 400, 172, 12]
bbox: [544, 364, 789, 447, 13]
bbox: [575, 126, 621, 159, 14]
bbox: [142, 119, 192, 159, 15]
bbox: [485, 141, 522, 189, 16]
bbox: [367, 118, 436, 162, 17]
bbox: [179, 122, 213, 150, 18]

0: 320x640 19 cars, 1 bus, 3 trucks, 1275.1ms
Speed: 2.6ms preprocess, 1275.1ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)
bbox: [174, 289, 342, 420, 0]
bbox: [516, 245, 661, 363, 1]
bbox: [484, 182, 577, 255, 5]
bbox: [4, 141, 91, 

KeyboardInterrupt: 

In [ ]:
!ls -l

total 103724
-rw-r--r-- 1 root root  4843566 Nov 27 12:54 cartraffic1.mp4
-rw-r--r-- 1 root root      699 Nov 27 12:53 coco.txt
-rw-r--r-- 1 root root 13570514 Nov 27 13:03 output.mp4
drwxr-xr-x 2 root root     4096 Nov 27 12:54 __pycache__
drwxr-xr-x 1 root root     4096 Nov 21 14:24 sample_data
-rw-r--r-- 1 root root     1797 Nov 27 12:53 tracker.py
-rw-r--r-- 1 root root 87769683 Nov 27 12:54 yolov8l.pt
